In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.0 MB/s eta 0:00:00


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import transformers
transformers.__version__

'4.27.1'

In [6]:
import pandas as pd
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [8]:
df = pd.read_csv("/content/gdrive/MyDrive/GTMSA/aap.csv")
df.head(1)

,paragraph,value,label,name,type,company,sent,entity,entity_type_ext,sentence,phrases,qa_temp,key,score,question,answer
0,As of the last business day of the registrant'...,"9,274,738,343",['Entity Public Float'],dei_EntityPublicFloat,monetaryItemType,aap,As of the last business day of the registrant’...,"$9,274,738,343",MONEY,As of the last business day of the registrant’...,"['last business day of registrant', 'aggregate...","['aggregate market value of common stock', 0.9...",aggregate market value of common stock,0.988797,what is aggregate market value of common stock ?,"$9,274,738,343"


In [23]:
max([len(i) for i in df.loc[:,'sent']])

948

In [18]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels = 3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [30]:
import torch
import numpy as np
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
#preprocess data
text = df["sent"].tolist()
labels = {0:'Negative',1:'Neutral',2:'Positive'}

# add the sentiment analysis results to the list
sentiment_labels = []
for sent in text[:50]:
    inputs = tokenizer(sent, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
   
    val = labels[np.argmax(outputs.detach().numpy())]
    sentiment_labels.append(val)
df['sentiment'] = pd.Series(sentiment_labels)

In [37]:
text = df["sent"].tolist()

In [35]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
para = df["paragraph"].tolist()
nlp(para[:10])

[{'label': 'Neutral', 'score': 0.9999982118606567},
 {'label': 'Neutral', 'score': 0.9999996423721313},
 {'label': 'Neutral', 'score': 0.9999896287918091},
 {'label': 'Neutral', 'score': 0.9999896287918091},
 {'label': 'Neutral', 'score': 0.9999896287918091},
 {'label': 'Neutral', 'score': 0.993798553943634},
 {'label': 'Neutral', 'score': 0.993798553943634},
 {'label': 'Neutral', 'score': 0.9999978542327881},
 {'label': 'Neutral', 'score': 0.9997281432151794},
 {'label': 'Neutral', 'score': 0.9997281432151794}]

In [38]:
# import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# load FINBERT model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# prepare the data and tokenize text to be sent to model
inputs = tokenizer(text[:10], padding = True, truncation = True, return_tensors='pt')
# run sentiment analysis
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
model.config.id2label
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()
table = {'Headline':text[:10], "Positive":positive, "Negative":negative, "Neutral":neutral}      
df_sentiment = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])
df_sentiment

,Headline,Positive,Negative,Neutral
0,As of the last business day of the registrant’...,0.017979,0.042359,0.939662
1,"As of February 17, 2021, the number of shares ...",0.018833,0.031181,0.949987
2,"As of January 2, 2021, our operations are comp...",0.036761,0.014995,0.948244
3,"As of January 2, 2021, our operations are comp...",0.036761,0.014995,0.948244
4,"In addition, we served 1,277 independently own...",0.114093,0.009234,0.876673
5,Total deferred vendor incentives recorded as a...,0.034242,0.913266,0.052492
6,Total deferred vendor incentives recorded as a...,0.034242,0.913266,0.052492
7,"We have five operating segments, defined as “N...",0.029658,0.018126,0.952216
8,The adoption of ASU 2016 02 resulted in the re...,0.051752,0.020187,0.928060
9,The adoption of ASU 2016 02 resulted in the re...,0.051752,0.020187,0.928060


In [39]:
label_list = ['Positive', 'Negative', 'Neutral']
df_sentiment['sentiment'] = df_sentiment.apply(lambda x: label_list[np.argmax([x.Positive,x.Negative,x.Neutral])], axis = 1)
df_sentiment

,Headline,Positive,Negative,Neutral,sentiment
0,As of the last business day of the registrant’...,0.017979,0.042359,0.939662,Neutral
1,"As of February 17, 2021, the number of shares ...",0.018833,0.031181,0.949987,Neutral
2,"As of January 2, 2021, our operations are comp...",0.036761,0.014995,0.948244,Neutral
3,"As of January 2, 2021, our operations are comp...",0.036761,0.014995,0.948244,Neutral
4,"In addition, we served 1,277 independently own...",0.114093,0.009234,0.876673,Neutral
5,Total deferred vendor incentives recorded as a...,0.034242,0.913266,0.052492,Negative
6,Total deferred vendor incentives recorded as a...,0.034242,0.913266,0.052492,Negative
7,"We have five operating segments, defined as “N...",0.029658,0.018126,0.952216,Neutral
8,The adoption of ASU 2016 02 resulted in the re...,0.051752,0.020187,0.928060,Neutral
9,The adoption of ASU 2016 02 resulted in the re...,0.051752,0.020187,0.928060,Neutral
